In [62]:
import numpy as np

def get_ddt(sbox):
    size = len(sbox)
    ddt = np.array([[0]*size]*size)
    for x in range(size):
        for a in range(size):
            b = sbox[x ^ a] ^ sbox[x]
            ddt[a, b] += 1
    return ddt

sboxes = {
'PRESENT':	[0xC, 5, 6, 0xB, 9, 0, 0xA, 0xD, 3, 0xE, 0xF, 8, 4, 7, 1, 2],
'Midori':	[0xC, 0xA, 0xD, 3, 0xE, 0xB, 0xF, 7, 8, 9, 1, 5, 0, 2, 4, 6],
'GIFT':		[1, 0xA, 4, 0xC, 6, 0xF, 3, 9, 2, 0xD, 0xB, 7, 5, 0, 8, 0xE],
'SKINNY1':	[0xC, 6, 9, 0, 1, 0xA, 2, 0xB, 3, 8, 5, 0xD, 4, 0xE, 7, 0xF],
'SKINNY2':	[3, 4, 6, 8, 0xC, 0xA, 1, 0xE, 9, 2, 5, 7, 0, 0xB, 0xD, 0xF],
'LILLIPUT':	[4, 8, 7, 1, 9, 3, 2, 0xE, 0, 0xB, 3, 0xF, 0xA, 5, 0xD, 0xC],
#'Ex': [2, 6, 1, 7, 3, 0, 4, 5]
}

#get_ddt(sboxes['Ex'])

In [1]:
from sys import version_info
version_info

sys.version_info(major=3, minor=4, micro=4, releaselevel='final', serial=0)

In [2]:
import numpy as np

In [3]:
present = [0xC, 5, 6, 0xB, 9, 0, 0xA, 0xD, 3, 0xE, 0xF, 8, 4, 7, 1, 2]
midori  = [0xC, 0xA, 0xD, 3, 0xE, 0xB, 0xF, 7, 8, 9, 1, 5, 0, 2, 4, 6]
gift    = [1, 0xA, 4, 0xC, 6, 0xF, 3, 9, 2, 0xD, 0xB, 7, 5, 0, 8, 0xE]
present[12]

4

In [58]:
def get_ddt(sbox):
    size = len(sbox)
    ddt = np.array([[0]*size]*size)
    for x in range(size):
        for a in range(size):
            b = sbox[x ^ a] ^ sbox[x]
            ddt[a, b] += 1
    return ddt

print(get_ddt(midori), '\n\n')#.max() / 2^16

[[16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  4  0  2  2  2  0  2  0  0  0  0  0  2  0]
 [ 0  4  0  0  4  0  0  0  0  4  0  0  4  0  0  0]
 [ 0  0  0  0  2  0  4  2  2  2  0  0  0  2  0  2]
 [ 0  2  4  2  2  2  0  0  2  0  0  2  0  0  0  0]
 [ 0  2  0  0  2  0  0  4  0  2  4  0  2  0  0  0]
 [ 0  2  0  4  0  0  0  2  2  0  0  0  2  2  0  2]
 [ 0  0  0  2  0  4  2  0  0  0  0  2  0  4  2  0]
 [ 0  2  0  2  2  0  2  0  0  2  0  2  2  0  2  0]
 [ 0  0  4  2  0  2  0  0  2  2  0  2  2  0  0  0]
 [ 0  0  0  0  0  4  0  0  0  0  4  0  0  4  0  4]
 [ 0  0  0  0  2  0  0  2  2  2  0  4  0  2  0  2]
 [ 0  0  4  0  0  2  2  0  2  2  0  0  2  0  2  0]
 [ 0  0  0  2  0  0  2  4  0  0  4  2  0  0  2  0]
 [ 0  2  0  0  0  0  0  2  2  0  0  0  2  2  4  2]
 [ 0  0  0  2  0  0  2  0  0  0  4  2  0  0  2  4]] 




In [6]:
def get_linear(param):
    masks = [0b1000, 0b0100, 0b0010, 0b0001]
    p = param
    return lambda x:((x&p&masks[0])>>3)^((x&p&masks[1])>>2)^((x&p&masks[2])>>1)^x&p&masks[3]
    
linears = {x: get_linear(x) for x in range(16)} 

In [7]:
masks = [0b1000, 0b0100, 0b0010, 0b0001]
def linear(x, param):
    return ((x&param&masks[0]) >> 3) ^ ((x&param&masks[1]) >> 2) ^ ((x&param&masks[2]) >> 1) ^ x&param&masks[3]

def correlations(f1, f2):
    out1 = int(''.join(['{}'.format(f1(x)) for x in range(16)]), 2)
    out2 = int(''.join(['{}'.format(f2(x)) for x in range(16)]), 2)
    return 16 - bin(out1 ^ out2).count('1') #2**(1-n)*ret - 1

correlations(linears[10], linears[0])

8

In [8]:
def get_lat(sbox):
    size = len(sbox)
    lat = np.array([[0]*size]*size)
    for a in range(size):
        for b in range(size):
            lat[a, b] = correlations(linears[a], lambda x: linears[b](sbox[x]))
    return lat

(get_lat(gift)-8)#[1, 9]#*(2**(1-4)) - 1).max()

array([[ 8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  2, -2, -2,  2,  4,  0,  0, -4, -2, -2, -2, -2],
       [ 0,  0,  0, -4,  0,  4,  0,  0,  2,  2,  2, -2,  2, -2,  2,  2],
       [ 0,  0,  0, -4, -2, -2,  2, -2,  2, -2, -2, -2,  0,  4,  0,  0],
       [ 0,  0,  0,  0,  0,  0, -4, -4,  0,  0,  0,  0,  0,  0,  4, -4],
       [ 0,  0,  0,  0,  2, -2,  2, -2,  0,  4,  4,  0,  2,  2, -2, -2],
       [ 0,  0,  0, -4,  4,  0,  0,  0, -2, -2, -2,  2,  2, -2, -2, -2],
       [ 0,  0,  0,  4,  2,  2,  2, -2,  2, -2, -2, -2,  4,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -4,  0, -4,  0,  0,  0,  0,  0, -4,  0,  4],
       [ 0,  0,  0,  0, -2, -2,  2,  2,  4,  0,  0,  4,  2, -2,  2, -2],
       [ 0,  0, -4,  0,  0,  0,  4,  0, -2, -2,  2, -2, -2, -2,  2, -2],
       [ 0,  0,  4,  0,  2, -2,  2,  2, -2,  2, -2, -2,  0,  0,  4,  0],
       [ 0,  4,  4,  0,  0,  0,  0,  0,  0, -4,  4,  0,  0,  0,  0,  0],
       [ 0, -4,  4,  0, -2,  2,  2, -2,  0,  0,  0,

In [63]:
sboxes = {
'PRESENT':	[0xC, 5, 6, 0xB, 9, 0, 0xA, 0xD, 3, 0xE, 0xF, 8, 4, 7, 1, 2],
'Midori':	[0xC, 0xA, 0xD, 3, 0xE, 0xB, 0xF, 7, 8, 9, 1, 5, 0, 2, 4, 6],
'GIFT':		[1, 0xA, 4, 0xC, 6, 0xF, 3, 9, 2, 0xD, 0xB, 7, 5, 0, 8, 0xE],
'SKINNY1':	[0xC, 6, 9, 0, 1, 0xA, 2, 0xB, 3, 8, 5, 0xD, 4, 0xE, 7, 0xF],
'SKINNY2':	[3, 4, 6, 8, 0xC, 0xA, 1, 0xE, 9, 2, 5, 7, 0, 0xB, 0xD, 0xF],
'LILLIPUT':	[4, 8, 7, 1, 9, 3, 2, 0xE, 0, 0xB, 3, 0xF, 0xA, 5, 0xD, 0xC],
}

get_ddt(sboxes['PRESENT'])
#get_lat(sboxes['LILLIPUT'])-8

array([[16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  4,  0,  0,  0,  4,  0,  4,  0,  0,  0,  4,  0,  0],
       [ 0,  0,  0,  2,  0,  4,  2,  0,  0,  0,  2,  0,  2,  2,  2,  0],
       [ 0,  2,  0,  2,  2,  0,  4,  2,  0,  0,  2,  2,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  4,  2,  2,  0,  2,  2,  0,  2,  0,  2,  0],
       [ 0,  2,  0,  0,  2,  0,  0,  0,  0,  2,  2,  2,  4,  2,  0,  0],
       [ 0,  0,  2,  0,  0,  0,  2,  0,  2,  0,  0,  4,  2,  0,  0,  4],
       [ 0,  4,  2,  0,  0,  0,  2,  0,  2,  0,  0,  0,  2,  0,  0,  4],
       [ 0,  0,  0,  2,  0,  0,  0,  2,  0,  2,  0,  4,  0,  2,  0,  4],
       [ 0,  0,  2,  0,  4,  0,  2,  0,  2,  0,  0,  0,  2,  0,  4,  0],
       [ 0,  0,  2,  2,  0,  4,  0,  0,  2,  0,  2,  0,  0,  2,  2,  0],
       [ 0,  2,  0,  0,  2,  0,  0,  0,  4,  2,  2,  2,  0,  2,  0,  0],
       [ 0,  0,  2,  0,  0,  4,  0,  2,  2,  2,  2,  0,  0,  0,  2,  0],
       [ 0,  2,  4,  2,  2,  0,  0,  2,  0,  0,  2,

In [10]:
difs = dict()
ddts = dict()
for name, sbox in sboxes.items():
    ddt = get_ddt(sbox)
    ddts[name] = ddt
    ddt[0, 0] = 0
    difs[name] = ddt.max() 
    ddt[0, 0] = 16
difs 

{'GIFT': 6,
 'LILLIPUT': 6,
 'Midori': 4,
 'PRESENT': 4,
 'SKINNY1': 4,
 'SKINNY2': 4}

In [89]:
def export_truth_table(filename, truth_table):
    with open(filename, 'w') as f:
        function_name = filename.split('.')[0]
        f.write('x1,x2,x3,x4,y1,y2,y3,y4,,{}\n'.format(function_name[:8])) # 8 is Logic Friday max len 
        for i in range(len(truth_table)):
            index = bin(i)[2:].zfill(8)
            index = ','.join([x for x in index])
            f.write(index + ',,{}\n'.format(truth_table[i]))

export_truth_table('Midori.csv', mddts['Midori'])

In [37]:
print(ddts['PRESENT'])

[[16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  4  0  4  0  0  0  4  0  0]
 [ 0  0  0  2  0  4  2  0  0  0  2  0  2  2  2  0]
 [ 0  2  0  2  2  0  4  2  0  0  2  2  0  0  0  0]
 [ 0  0  0  0  0  4  2  2  0  2  2  0  2  0  2  0]
 [ 0  2  0  0  2  0  0  0  0  2  2  2  4  2  0  0]
 [ 0  0  2  0  0  0  2  0  2  0  0  4  2  0  0  4]
 [ 0  4  2  0  0  0  2  0  2  0  0  0  2  0  0  4]
 [ 0  0  0  2  0  0  0  2  0  2  0  4  0  2  0  4]
 [ 0  0  2  0  4  0  2  0  2  0  0  0  2  0  4  0]
 [ 0  0  2  2  0  4  0  0  2  0  2  0  0  2  2  0]
 [ 0  2  0  0  2  0  0  0  4  2  2  2  0  2  0  0]
 [ 0  0  2  0  0  4  0  2  2  2  2  0  0  0  2  0]
 [ 0  2  4  2  2  0  0  2  0  0  2  2  0  0  0  0]
 [ 0  0  2  2  0  0  2  2  2  2  0  0  2  2  0  0]
 [ 0  4  0  0  4  0  0  0  0  0  0  0  0  0  4  4]]


In [14]:
def milpify_ddt(ddt):
    truth_table = [0] * 256
    for i in range(16):
        for j in range(16):
            if ddt[i, j] > 0:
                truth_table[(i << 4) + j] = 1
    return truth_table

mddts = dict()
for name, sbox in ddts.items():
    mddts[name] = milpify_ddt(ddts[name])
mddts['PRESENT']

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,


In [28]:
bin((8 << 4) + 15)

'0b10001111'

In [11]:
lins = dict()
for name, sbox in sboxes.items():
    lat = get_lat(sbox)
    lat[0, 0] = 0
    lins[name] = lat.max() 
lins

{'GIFT': 12,
 'LILLIPUT': 13,
 'Midori': 12,
 'PRESENT': 12,
 'SKINNY1': 12,
 'SKINNY2': 12}

In [12]:
'0110'.zfill(7)

'0000110'

In [13]:
from math import log2
def get_deg_by_polynom(s):
    x = ['1', 'x4', 'x3', 'x3x4', 'x2', 'x2x4', 'x2x3', 'x2x3x4', 'x1', 'x1x4', 'x1x3', 'x1x3x4', 'x1x2', 'x1x2x4', 'x1x2x3', 'x1x2x3x4']
    polynom = []
    for i in range(16):
        if s[i] == '1':
            polynom.append(x[i])
    degree = max([len(x)//2 for x in polynom])
    xs = ['x1', 'x2', 'x3', 'x4']
    lin = [x for x in xs if x in polynom] 
    non_lin ={x for x in xs for m in polynom if x in m and len(x) < len(m)}
    non_lin = [x for x in lin if x not in non_lin]
    return degree, non_lin, ' ^ '.join(polynom)

def get_bf_degree(table):
    n = int(log2(len(table)))
    t = [''] * (n + 1)
    t[0] = table
    for row in range(n):
        cell_size = 2**row
        next_row = 0
        for cell in range(0, 2**n, cell_size*2):
            left = int(t[row][cell: cell + cell_size], 2)
            right = int(t[row][cell + cell_size: cell + 2*cell_size], 2)
            t[row + 1] += bin(left)[2:].zfill(cell_size) + bin(left^ right)[2:].zfill(cell_size)
    
    
    return get_deg_by_polynom(t[n]), t[n]

def get_sbox_degree(sbox):
    masks = [0b1000, 0b0100, 0b0010, 0b0001]
    t = [''] * 4
    for i in range(4):
        table = ''.join([str((masks[i] & sbox[x]) >> (3 - i)) for x in range(16)])
        (degree, non_lin, polynom), t[i] = get_bf_degree(table)
        print(i + 1, degree, non_lin, polynom)
    
    

for name, sbox in sboxes.items():
    print(name)
    get_sbox_degree(sboxes[name])


PRESENT
1 3 [] 1 ^ x4 ^ x3 ^ x2x3 ^ x2x3x4 ^ x1 ^ x1x3x4 ^ x1x2x4
2 3 [] 1 ^ x3x4 ^ x2 ^ x1 ^ x1x4 ^ x1x3 ^ x1x3x4 ^ x1x2x4
3 3 [] x3 ^ x2x3x4 ^ x1 ^ x1x3 ^ x1x3x4 ^ x1x2 ^ x1x2x4
4 2 ['x1', 'x4'] x4 ^ x2 ^ x2x3 ^ x1
SKINNY1
1 2 ['x4'] 1 ^ x4 ^ x2 ^ x1 ^ x1x2
2 2 ['x1'] 1 ^ x3 ^ x2 ^ x2x3 ^ x1
3 3 [] x4 ^ x3x4 ^ x2x3 ^ x1 ^ x1x3 ^ x1x2 ^ x1x2x3
4 3 [] x3 ^ x3x4 ^ x2 ^ x2x4 ^ x2x3x4 ^ x1 ^ x1x4 ^ x1x3 ^ x1x2x3
LILLIPUT
1 3 [] x4 ^ x3x4 ^ x2 ^ x2x3 ^ x2x3x4 ^ x1x3x4 ^ x1x2x3
2 4 [] 1 ^ x4 ^ x2 ^ x2x4 ^ x2x3x4 ^ x1 ^ x1x4 ^ x1x3x4 ^ x1x2 ^ x1x2x3 ^ x1x2x3x4
3 3 [] x3 ^ x3x4 ^ x2x4 ^ x1x4 ^ x1x2 ^ x1x2x4
4 4 [] x3 ^ x2 ^ x1x4 ^ x1x3x4 ^ x1x2 ^ x1x2x3x4
SKINNY2
1 3 [] x3x4 ^ x2 ^ x2x3 ^ x1 ^ x1x4 ^ x1x3 ^ x1x2x3
2 2 ['x3'] x4 ^ x3 ^ x2 ^ x1x4 ^ x1x2
3 2 ['x4'] 1 ^ x4 ^ x2 ^ x1 ^ x1x2
4 3 [] 1 ^ x4 ^ x3 ^ x3x4 ^ x2 ^ x2x4 ^ x1x3 ^ x1x2x4 ^ x1x2x3
Midori
1 3 [] 1 ^ x3x4 ^ x1x3 ^ x1x3x4 ^ x1x2 ^ x1x2x3
2 3 [] 1 ^ x4 ^ x2x3x4 ^ x1 ^ x1x4 ^ x1x3x4 ^ x1x2x3
3 2 [] x4 ^ x2 ^ x2x4 ^ x1x4 ^ x1x2
4 3

In [73]:
def avalanche(sbox):
    masks = [0b1000, 0b0100, 0b0010, 0b0001]
    res = []
    for mask in masks:
        sums =[0] * 4
        for x in range(16):
            s = '{0:04b}'.format(sbox[x] ^ sbox[x ^ mask])
            for i in range(4):
                if s[i] == '1':
                    sums[i] += 1
            #print(s)
        res.append(sums)
    return res

for name, sbox in sboxes.items():
    print(name)
    a = np.array(avalanche(sboxes[name]))
    print(a.min(), a.max())

PRESENT
8 16
GIFT
4 16
SKINNY2
0 16
LILLIPUT
4 14
SKINNY1
0 16
Midori
0 8


In [134]:
a = ['x1', 'x2', 'x3', 'x4']
x = ['{0:04b}'.format(i) for i in range(16)]
ms = []
for i in range(16):
    m = ''
    for j in range(4):
        if x[i][j] == '1':
            m += a[j]
    ms.append(m)
ms

['',
 'x4',
 'x3',
 'x3x4',
 'x2',
 'x2x4',
 'x2x3',
 'x2x3x4',
 'x1',
 'x1x4',
 'x1x3',
 'x1x3x4',
 'x1x2',
 'x1x2x4',
 'x1x2x3',
 'x1x2x3x4']

In [135]:
def hamming_weight(num):
    s = '{0:04b}'.format(num)
    return s.count('1')

def get_bn(sbox):
    min_hw = 9999
    for a in range(16):
        for b in range(16):
            if a != b:
                hw = hamming_weight(a ^ b) + hamming_weight(sbox[a] ^ sbox[b])
                if hw < min_hw:
                    min_hw = hw
    return min_hw

for name, sbox in sboxes.items():
    print(name, get_bn(sboxes[name]))

GIFT 2
PRESENT 3
SKINNY1 2
Midori 2
LILLIPUT 2
SKINNY2 2


In [81]:
def get_pb_ddts(ddt):
    ddt[0, 0] = 0
    pb_ddts = {x:0 for x in range(2, ddt.max() + 1, 2)}
    for pb in pb_ddts.keys():
        pb_ddt = np.array([[0]*16]*16)
        for i in range(16):
            for j in range(16):
                if ddt[i, j] == pb:
                    pb_ddt[i, j] = 1
        pb_ddts[pb] = pb_ddt
    return pb_ddts

pb_ddts = dict()
for alg, sbox in sboxes.items():
    pb_ddts[alg] = get_pb_ddts(ddts[alg])
    
pb_ddts['Midori']

{2: array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
        [0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0]]),
 4: array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      

In [146]:
def pp(a):
    new_a=[]
    for i in range(0,64):
        new_a.append(0)
    for i in range(0,16):
        new_a[i]=a[4*i]
    for i in range(0,16):
        new_a[i+16]=a[1+4*i]
    for i in range(0,16):
        new_a[i+32]=a[2+4*i]
    for i in range(0,16):
        new_a[i+48]=a[3+4*i]
    a=new_a
    return a

print(pp(list(range(64))))

[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 1, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 57, 61, 2, 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54, 58, 62, 3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 55, 59, 63]


In [153]:
64/16

4.0

In [163]:
def present_perm(i):
    r = i // 4
    q = i % 4
    #return 16*q+r 
    return 16*q+4*(r//4)+(3-r-q)%4

p = [present_perm(x) for x in range(64)]
print(p)
print(sorted(p))
mp = []
for i in range(16):
    mp += p[i::16]
print(mp)

[3, 18, 33, 48, 2, 17, 32, 51, 1, 16, 35, 50, 0, 19, 34, 49, 7, 22, 37, 52, 6, 21, 36, 55, 5, 20, 39, 54, 4, 23, 38, 53, 11, 26, 41, 56, 10, 25, 40, 59, 9, 24, 43, 58, 8, 27, 42, 57, 15, 30, 45, 60, 14, 29, 44, 63, 13, 28, 47, 62, 12, 31, 46, 61]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]
[3, 7, 11, 15, 18, 22, 26, 30, 33, 37, 41, 45, 48, 52, 56, 60, 2, 6, 10, 14, 17, 21, 25, 29, 32, 36, 40, 44, 51, 55, 59, 63, 1, 5, 9, 13, 16, 20, 24, 28, 35, 39, 43, 47, 50, 54, 58, 62, 0, 4, 8, 12, 19, 23, 27, 31, 34, 38, 42, 46, 49, 53, 57, 61]


In [166]:
x = [0]*64
x[3] = 1
new_x = [0]*64
for i in range(64):
    new_x[p[i]] = x[i]
    
str(new_x).replace(', ','')
new_x[48:52]

[1, 0, 0, 0]

In [111]:
def get_important(ddt):
    indexes = [1, 2, 4, 8]
    imp_ddt = np.array([[0]*4]*4)
    for i in range(4):
        for j in range(4):
            imp_ddt[i,j] = ddt[indexes[i],indexes[j]]
    return imp_ddt
            
for alg, sbox in sboxes.items():
    print(alg)
    print(get_important(ddts[alg]))


PRESENT
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
GIFT
[[0 0 0 2]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
SKINNY2
[[0 4 0 0]
 [0 0 4 0]
 [0 0 0 4]
 [4 0 0 0]]
LILLIPUT
[[2 0 0 0]
 [0 0 2 0]
 [0 0 0 0]
 [0 2 4 0]]
SKINNY1
[[0 0 0 4]
 [4 0 0 0]
 [0 4 0 0]
 [0 0 4 0]]
Midori
[[2 4 2 2]
 [4 0 4 0]
 [2 4 2 2]
 [2 0 2 0]]


In [101]:
ddts['Midori']

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 4, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0],
       [0, 4, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 4, 2, 2, 2, 0, 0, 0, 2, 0, 2],
       [0, 2, 4, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0],
       [0, 2, 0, 0, 2, 0, 0, 4, 0, 2, 4, 0, 2, 0, 0, 0],
       [0, 2, 0, 4, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 2],
       [0, 0, 0, 2, 0, 4, 2, 0, 0, 0, 0, 2, 0, 4, 2, 0],
       [0, 2, 0, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0],
       [0, 0, 4, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 4, 0, 4],
       [0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 4, 0, 2, 0, 2],
       [0, 0, 4, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0],
       [0, 0, 0, 2, 0, 0, 2, 4, 0, 0, 4, 2, 0, 0, 2, 0],
       [0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 4, 2],
       [0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 4, 2, 0, 0, 2, 4]])

In [90]:
for pb, ddt in pb_ddts['Midori'].items():
    export_truth_table('pb{}_Midori.csv'.format(pb), milpify_ddt(pb_ddts['Midori'][pb]))

In [12]:
def print_boolean_table(f, n=4):
    for x in range(2**n):
        print("%{}s|%s".format(n) % (bin(x)[2:], f(x)))

In [33]:
print_boolean_table(linears[1])

   0|0
   1|1
  10|0
  11|1
 100|0
 101|1
 110|0
 111|1
1000|0
1001|1
1010|0
1011|1
1100|0
1101|1
1110|0
1111|1


In [19]:
x = '11110000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000'
len(x)
print(x[:64])
print(x[64:])

1111000000000000000000000000000000000000000000000000000000000000
1000000000000000000000000000000000000000000000000000000000000000


In [24]:
from datetime import datetime
print(str(datetime.now()))


a = datetime.now()

2018-05-02 02:18:50.365248


In [45]:
s = '00000000000000100000000000010000000000000010000000000010000000000000000001000000000000000000001000000001000000000100000000000000'
len(s)
for i in range(0, 128, 16):
    print(i // 16, s[i:i+16])

0 0000000000000010
1 0000000000010000
2 0000000000100000
3 0000001000000000
4 0000000001000000
5 0000000000000010
6 0000000100000000
7 0100000000000000


In [37]:
x = '0000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000001000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000'
s = '000000100000000000000000000001000001000000000000100000000000000000000000100000000010000000000000'


print(len(x))
print(x[:64])
#print(x[64:128])
print(x[448-64:])

for i in range(64,128):
    if x[i] == '1':
        #print(i - 64)
        pass
    

448
0000000000000000000000000001000000000000000000000000000000000000
0000000010000000000000000000000000000000000000000000000000000000
